In [1]:
# Simple analysis script to calculate the total mass of gas implied by NH2 map
# and percentage at positive vs. negative longitudes

# All maps / positions double checked by eye 

In [3]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
#from matplotlib import *
#import aplpy
from astropy.io import fits
import numpy as np
from astropy import units as u

from astropy.wcs import WCS
from astropy.nddata.utils import Cutout2D
from astropy.coordinates import SkyCoord  # High-level coordinates
from astropy.coordinates import ICRS, Galactic, FK4, FK5  # Low-level frames
from astropy.coordinates import Angle, Latitude, Longitude  # Angles

from astropy.io import ascii
from astropy.table import Table

In [6]:
#path='/Users/battersby/Dropbox/Work/higal_cmz/nh2_pdfs/all_fits_files/'
path='/Users/cab16109/Dropbox/Work/higal_cmz/PAPER/FINAL_DATA_files_for_release/'

In [8]:
# Open column density file (standard analysis file, inner 7 x 2 deg)
#columnlist = fits.open(path+'column_properunits_conv36_source_only.fits')
columnlist = fits.open(path+'higalcmz_column_density_source_only_inner7deg.fits')
columnlist[0].data[np.where(columnlist[0].data==0.0)]=np.nan
column = columnlist[0].data
colhdr = columnlist[0].header

In [15]:
# Calculating total mass and save it as 'column_properunits_conv36_source_only_MASS_MAP.fits'
# Mass per pixel = column density per pixel [molecules cm^-2] * pixel area [cm^2] * mu * mh [kg] * 1/Msun [kg] = [Msun] 
# Total Mass = Sum(mass per pixel)

mu = 2.8 # Kauffman et al. 2008 appendix (mean molecular weight of molecular gas)
mh = 1.67e-27 # kg
msun = 1.99e30 #kg
distance = 8150 #pc Reid 2019 udpated
dcm = distance*3.086e18 # distance in cm

# Get pixel area in square centimeter
pixx_cm = dcm*(np.absolute(colhdr['CDELT1'])*3600./206265) # distance in cm * angle in radians
pixy_cm = dcm*(np.absolute(colhdr['CDELT2'])*3600./206265)
pix_area_cm2 = pixx_cm*pixy_cm

#gets confused if you try to do it all in one step since numbers BIG, so separate out constant factors.
column_to_mass_factor = (pix_area_cm2 * mu * mh) / msun
mass_map = column * column_to_mass_factor

#Write to a fits file
fits.writeto(path+'MASS_MAPS/higalcmz_column_density_source_only_inner7deg_MASS_MAP.fits', data=mass_map, header=colhdr, overwrite=True)

In [16]:
# Short script to save cutouts as fits file based on given position and size
# From: https://docs.astropy.org/en/stable/nddata/utils.html#cutout-images
def savecutout(filename, position, size, outfile):
    # Load the image and the WCS
    hdu = fits.open(filename)[0]
    wcs = WCS(hdu.header)

    # Make the cutout, including the WCS
    cutout = Cutout2D(hdu.data, position=position, size=size, wcs=wcs)

    # Put the cutout image in the FITS HDU
    hdu.data = cutout.data

    # Update the FITS header with the cutout WCS
    hdu.header.update(cutout.wcs.to_header())

    # Write the cutout to a new FITS file
    cutout_filename = outfile
    hdu.writeto(cutout_filename, overwrite=True)

In [20]:
# Restrict mass estimate to other fields, save mass map to check, and print out total mass

def print_region_mass(filename, outstring, position, size):
    outfile = path+'MASS_MAPS/mass_maps' +outstring+'.fits'
    savecutout(filename, position, size, outfile)
    hdu = fits.open(outfile)
    total_mass = np.nansum(hdu[0].data)
    print('Total Mass [Msun]: {:0.2e} over'.format(total_mass), outstring)

# Full mass map:
filename = path+'MASS_MAPS/higalcmz_column_density_source_only_inner7deg_MASS_MAP.fits'
    
# Total mass of full map:
hdu = fits.open(filename)
total_mass = np.nansum(hdu[0].data)
print('Total Mass [Msun]: {:0.2e} over'.format(total_mass), 'the inner 7 x 2.5 degrees (full map)')

# Save cutouts for each sub-region and print total mass

# Field 0 inner 7 deg [l] x 2 deg [b], full, positive, and negative maps
print_region_mass(filename, '_field0', 
                  SkyCoord(frame="galactic", l="0.0d", b="+0.0d"), 
                  u.Quantity([2,7], u.deg))  

print_region_mass(filename, '_field0pos', 
                  SkyCoord(frame="galactic", l="1.75d", b="+0.0d"), 
                  u.Quantity([2,3.5], u.deg)) 

print_region_mass(filename, '_field0neg', 
                  SkyCoord(frame="galactic", l="-1.75d", b="+0.0d"), 
                  u.Quantity([2,3.5], u.deg)) 

# Field 1 inner 4 deg [l] x 2 deg [b], full, positive, and negative maps.
print_region_mass(filename, '_field1', 
                  SkyCoord(frame="galactic", l="0.0d", b="+0.0d"), 
                  u.Quantity([2,4], u.deg))  

print_region_mass(filename, '_field1pos', 
                  SkyCoord(frame="galactic", l="1.0d", b="+0.0d"), 
                  u.Quantity([2,2], u.deg)) 

print_region_mass(filename, '_field1neg', 
                  SkyCoord(frame="galactic", l="-1.0d", b="+0.0d"), 
                  u.Quantity([2,2], u.deg)) 

# Field 2 inner 2 deg [l] x 2 deg [b], full, positive, and negative maps.
print_region_mass(filename, '_field2', 
                  SkyCoord(frame="galactic", l="0.0d", b="+0.0d"), 
                  u.Quantity([2,2], u.deg)) 

print_region_mass(filename, '_field2pos', 
                  SkyCoord(frame="galactic", l="0.5d", b="+0.0d"), 
                  u.Quantity([2,1], u.deg))

print_region_mass(filename, '_field2neg', 
                  SkyCoord(frame="galactic", l="-0.5d", b="+0.0d"), 
                  u.Quantity([2,1], u.deg))

# Field 3: Inner 1.05 deg x 1.05deg, full, positive, and negative maps.
print_region_mass(filename, '_field3', 
                  SkyCoord(frame="galactic", l="0.0d", b="+0.0d"), 
                  u.Quantity([1.05, 1.05], u.deg)) 

print_region_mass(filename, '_field3pos', 
                  SkyCoord(frame="galactic", l="0.2625d", b="+0.0d"), 
                  u.Quantity([1.05,0.525], u.deg))

print_region_mass(filename, '_field3neg', 
                  SkyCoord(frame="galactic", l="-0.2625d", b="+0.0d"), 
                  u.Quantity([1.05,0.525], u.deg))

# Field 4: l = -1.05 to 2.25, b = \pm 0.75, full only:
print_region_mass(filename, '_field4', 
                  SkyCoord(frame="galactic", l="0.6d", b="+0.0d"), 
                  u.Quantity([1.5, 3.3], u.deg))

# Field 5: l = -1.0 to 1.7, b= \pm 0.5, full only:
print_region_mass(filename, '_field5', 
                  SkyCoord(frame="galactic", l="0.35d", b="+0.0d"), 
                  u.Quantity([1.0, 2.7], u.deg))

# Field 6: l = -1.3 to 1.8, b=\pm 0.75, full only: 
print_region_mass(filename,'_field6',
                  SkyCoord(frame="galactic", l="0.25d", b="+0.0d"), 
                  u.Quantity([1.5, 3.1], u.deg))



Total Mass [Msun]: 2.86e+07 over the inner 7 x 2.5 degrees (full map)
Total Mass [Msun]: 2.84e+07 over _field0
Total Mass [Msun]: 2.09e+07 over _field0pos
Total Mass [Msun]: 7.48e+06 over _field0neg
Total Mass [Msun]: 2.39e+07 over _field1
Total Mass [Msun]: 1.77e+07 over _field1pos
Total Mass [Msun]: 6.25e+06 over _field1neg
Total Mass [Msun]: 1.48e+07 over _field2
Total Mass [Msun]: 1.02e+07 over _field2pos
Total Mass [Msun]: 4.60e+06 over _field2neg
Total Mass [Msun]: 6.03e+06 over _field3
Total Mass [Msun]: 3.39e+06 over _field3pos
Total Mass [Msun]: 2.64e+06 over _field3neg
Total Mass [Msun]: 2.26e+07 over _field4
Total Mass [Msun]: 2.03e+07 over _field5
Total Mass [Msun]: 2.24e+07 over _field6


In [21]:
# Calculate positive vs. negative for each
# Field 0
print('Field 0 positive percentage:',(2.09e7/2.84e7)*100)
print('Field 0 negative percentage:',(0.748e7/2.84e7)*100)
# Field 1
print('Field 1 positive percentage:',(1.77e7/2.39e7)*100)
print('Field 1 negative percentage:',(0.625e7/2.39e7)*100)
# Field 2
print('Field 2 positive percentage:',(1.02e7/1.48e7)*100)
print('Field 2 negative percentage:',(0.460e7/1.48e7)*100)
# Field 3
print('Field 3 positive percentage:',(0.339e7/0.603e7)*100)
print('Field 3 negative percentage:',(0.264e7/0.603e7)*100)


Field 0 positive percentage: 73.59154929577466
Field 0 negative percentage: 26.338028169014084
Field 1 positive percentage: 74.05857740585773
Field 1 negative percentage: 26.15062761506276
Field 2 positive percentage: 68.91891891891892
Field 2 negative percentage: 31.08108108108108
Field 3 positive percentage: 56.21890547263681
Field 3 negative percentage: 43.78109452736319


In [25]:
# Now compare with no bg sub version:
# Full COLUMN map:
colfile = path+'higalcmz_WITH_BG_column_density_inner40deg.fits'
# Open column density file (Full inner 40deg, WITH BG)
columnlist = fits.open(colfile)
columnlist[0].data[np.where(columnlist[0].data==0.0)]=np.nan
colhdr = columnlist[0].header
# Get pixel area in square centimeter
pixx_cm = dcm*(np.absolute(colhdr['CDELT1'])*3600./206265) # distance in cm * angle in radians
pixy_cm = dcm*(np.absolute(colhdr['CDELT2'])*3600./206265)
pix_area_cm2 = pixx_cm*pixy_cm

#gets confused if you try to do it all in one step since numbers BIG, so separate out constant factors.
column_to_mass_factor = (pix_area_cm2 * mu * mh) / msun
mass_map = column * column_to_mass_factor

#Write to a fits file
fits.writeto(path+'MASS_MAPS/higalcmz_WITH_BG_column_density_inner40deg_MASS_MAP.fits', data=mass_map, header=colhdr, overwrite=True)


#Full MASS map:
#filename=path+'MASS_MAPS/higalcmz_WITH_BG_column_density_inner40deg_MASS_MAP.fits'
filename=path+'MASS_MAPS/higalcmz_WITH_BG_column_density_inner40deg_MASS_MAP.fits'
print('WITH BACKGROUND')

# Field 0 inner 7 deg [l] x 2 deg [b], full, positive, and negative maps
print_region_mass(filename, '_field0WBG', 
                  SkyCoord(frame="galactic", l="0.0d", b="+0.0d"), 
                  u.Quantity([2,7], u.deg))  

print_region_mass(filename, '_field0posWBG', 
                  SkyCoord(frame="galactic", l="1.75d", b="+0.0d"), 
                  u.Quantity([2,3.5], u.deg)) 

print_region_mass(filename, '_field0negWBG', 
                  SkyCoord(frame="galactic", l="-1.75d", b="+0.0d"), 
                  u.Quantity([2,3.5], u.deg)) 

# Field 1 inner 4 deg [l] x 2 deg [b], full, positive, and negative maps.
print_region_mass(filename, '_field1WBG', 
                  SkyCoord(frame="galactic", l="0.0d", b="+0.0d"), 
                  u.Quantity([2,4], u.deg))  

print_region_mass(filename, '_field1posWBG', 
                  SkyCoord(frame="galactic", l="1.0d", b="+0.0d"), 
                  u.Quantity([2,2], u.deg)) 

print_region_mass(filename, '_field1negWBG', 
                  SkyCoord(frame="galactic", l="-1.0d", b="+0.0d"), 
                  u.Quantity([2,2], u.deg)) 

# Field 2 inner 2 deg [l] x 2 deg [b], full, positive, and negative maps.
print_region_mass(filename, '_field2WBG', 
                  SkyCoord(frame="galactic", l="0.0d", b="+0.0d"), 
                  u.Quantity([2,2], u.deg)) 

print_region_mass(filename, '_field2posWBG', 
                  SkyCoord(frame="galactic", l="0.5d", b="+0.0d"), 
                  u.Quantity([2,1], u.deg))

print_region_mass(filename, '_field2negWBG', 
                  SkyCoord(frame="galactic", l="-0.5d", b="+0.0d"), 
                  u.Quantity([2,1], u.deg))

# Field 3: Inner 1.05 deg x 1.05deg, full, positive, and negative maps.
print_region_mass(filename, '_field3WBG', 
                  SkyCoord(frame="galactic", l="0.0d", b="+0.0d"), 
                  u.Quantity([1.05, 1.05], u.deg)) 

print_region_mass(filename, '_field3posWBG', 
                  SkyCoord(frame="galactic", l="0.2625d", b="+0.0d"), 
                  u.Quantity([1.05,0.525], u.deg))

print_region_mass(filename, '_field3negWBG', 
                  SkyCoord(frame="galactic", l="-0.2625d", b="+0.0d"), 
                  u.Quantity([1.05,0.525], u.deg))

# Field 4: l = -1.05 to 2.25, b = \pm 0.75, full only:
print_region_mass(filename, '_field4WBG', 
                  SkyCoord(frame="galactic", l="0.6d", b="+0.0d"), 
                  u.Quantity([1.5, 3.3], u.deg))

# Field 5: l = -1.0 to 1.7, b= \pm 0.5, full only:
print_region_mass(filename, '_field5WBG', 
                  SkyCoord(frame="galactic", l="0.35d", b="+0.0d"), 
                  u.Quantity([1.0, 2.7], u.deg))

# Field 6: l = -1.3 to 1.8, b=\pm 0.75, full only: 
print_region_mass(filename,'_field6WBG',
                  SkyCoord(frame="galactic", l="0.25d", b="+0.0d"), 
                  u.Quantity([1.5, 3.1], u.deg))



WITH BACKGROUND
Total Mass [Msun]: 7.79e+07 over _field0WBG
Total Mass [Msun]: 4.56e+07 over _field0posWBG
Total Mass [Msun]: 3.23e+07 over _field0negWBG
Total Mass [Msun]: 5.05e+07 over _field1WBG
Total Mass [Msun]: 3.03e+07 over _field1posWBG
Total Mass [Msun]: 2.01e+07 over _field1negWBG
Total Mass [Msun]: 2.80e+07 over _field2WBG
Total Mass [Msun]: 1.63e+07 over _field2posWBG
Total Mass [Msun]: 1.16e+07 over _field2negWBG
Total Mass [Msun]: 1.02e+07 over _field3WBG
Total Mass [Msun]: 5.31e+06 over _field3posWBG
Total Mass [Msun]: 4.85e+06 over _field3negWBG
Total Mass [Msun]: 4.08e+07 over _field4WBG
Total Mass [Msun]: 3.08e+07 over _field5WBG
Total Mass [Msun]: 3.92e+07 over _field6WBG


In [32]:
# Calculate positive vs. negative for each
print('WITH BACKGROUND')
# Field 0
print('Field 0 positive percentage:',(4.56e7/7.79e7)*100)
print('Field 0 negative percentage:',(3.23e7/7.79e7)*100)
# Field 1
print('Field 1 positive percentage:',(3.03e7/5.05e7)*100)
print('Field 1 negative percentage:',(2.01e7/5.05e7)*100)
# Field 2
print('Field 2 positive percentage:',(1.63e7/2.80e7)*100)
print('Field 2 negative percentage:',(1.16e7/2.80e7)*100)
# Field 3
print('Field 3 positive percentage:',(0.531e7/1.02e7)*100)
print('Field 3 negative percentage:',(0.485e7/1.02e7)*100)

WITH BACKGROUND
Field 0 positive percentage: 58.536585365853654
Field 0 negative percentage: 41.46341463414634
Field 1 positive percentage: 60.0
Field 1 negative percentage: 39.801980198019805
Field 2 positive percentage: 58.214285714285715
Field 2 negative percentage: 41.42857142857143
Field 3 positive percentage: 52.05882352941177
Field 3 negative percentage: 47.549019607843135
